<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# IRR Filters

In [1]:
import math as ma
import cmath as cm

Butterworth

In [2]:
########## GENERAL IRR ##############

def Rp(delP):
    rp = -20*ma.log10((1-delP)/(1+delP))
    return rp

def As(delS,delP):
    ds = -20*ma.log10(delS/(1+delP))
    return ds

def omegaP(Wp):
    wp = ma.tan(Wp/2)
    return wp
  
def omegaS(Ws):
    ws = ma.tan(Ws/2)
    return ws

def d_param(delP,delS):
    d = (((1-delP)**(-2)-1)/(delS**(-2)-1))**(1/2)
    return d

def k_param(omega_p,omega_s):
    k = omega_p/omega_s
    return k
############# BUTTERWORTH #####################    
# get the butterworth coefficients for
# N <= 8
def getButtCo(N):
    if ((N > 8) or (N != int(N)) or (N < 1)):
            print("Enter an Integer between 1 and 8")       
    else:
        if N == 1:
            return 1.0
        if N == 2:
            return [1.4142, 1]
        if N == 3:
            return [2.0, 2.0, 1.0]
        if N == 4:
            return [2.6131, 3.4142,2.6131, 1.0]
        if N == 5:
            return [3.2361, 5.2361, 5.2361, 3.2361,1]
        if N == 6:
            return [3.8637, 7.4641, 9.1416, 7.4641, 3.8637,1.0]
        if N == 7:
            return [4.4940, 10.0978, 14.5918, 14.5918, 10.0978, 4.4940,1.0]
        if N == 8:
            return [5.1258,13.1371,21.8462,25.6884,21.8462,13.1372,5.1258,1.0]
            
def buttord(d,k):
    n = ma.ceil(ma.log(d)/ma.log(k))
    return n

def setWc(Wp,Ws):
      return (Wp+Ws)/2

    
############## CHEBYSHEV ################

def chebord(d,k):
    di = 1/d
    ki = 1/k
    num = ma.log(di+ma.sqrt(di**2+1))
    dem = ma.log(ki+ma.sqrt(ki**2+1))
    n = ma.ceil(num/dem)
    return n

def getChebE(dP):
      return ((1-dP)**(-2)-1)**(1/2)

def getTN(N):
    tn = []
    k = list(range(0,len(N)))
    
############## ELLIPTICAL ###############

def ellipord(d,k):
    num = 1-(1-k**2)**(1/4)
    dem = 1+(1-k**2)**(1/4)
    q = 0.5*(num/dem)
    k = 16/(d**2)
    q0 = 1/q
    n = ma.ceil(ma.log(k)/ma.log(q0))
    return n



In [3]:
wp = 0.375*ma.pi
ws = 0.5*ma.pi

In [4]:
Wp = omegaP(wp)
Ws = omegaS(ws)
Wc = setWc(Wp,Ws)

In [5]:
dp = 0.01
ds = dp
d = d_param(dp,ds)
k = k_param(Wp,Ws)
N = buttord(d,k)

In [6]:
# FFT for some digital signal x
# using Cooley-Turkey 

def fft(x,form=None):
    OP = 0
    N = len(x)
    if N == 1:
        return [x[0]]
    X = [0] * N
    even = fft(x[:N:2])
    odd = fft(x[1:N:2])
    for k in range(N//2):
        w = ma.e**(-2j*ma.pi * k/N)
        X[k] = even[k] + w * odd[k]
        X[k + N//2] = even[k] - w * odd[k]
        OP += 2
    if form=='Magnitude':
        return [abs(i) for i in X]
    if form=='Phase':
        return [cm.phase(i) for i in X]
    if form=='Real':
        return [i.real for i in X]
    if form=='Imaginary':
        return [i.imag for i in X]
    return X

In [7]:
x = [0,1,1,2,1,3]
X = fft(x)
r = [i.real for i in X]
i = [i.imag for i in X]
mag = [abs(i) for i in X]
pha = [cm.phase(i) for i in X]

In [8]:
Xmag = fft(x,'Magnitude')

In [9]:
Xmag

[8.0, 2.0, 0.0, 4.0, 2.0, 0.0]